<a href="https://colab.research.google.com/github/MattWang-Portfolio/MattWang-Portfolio-data-processing-algorithm-Skin_Cancer/blob/main/skincancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Skin Cancer MNIST: HAM10000 https://www.kaggle.com/kmader/skin-cancer-mnist-ham10000



In [2]:
#keras.utils: 做one-hot encoding用
#sklearn.model_selection: 分割訓練集和測試集
#os: 用來建立檔案、刪除檔案
#PIL: (圖像處理庫)匯入圖像
#seed: 設定種子，使每次隨機產生的資料有相同結果。可將數字改成自己的學號(或其他數字)
import numpy as np
import pandas as pd
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import os
from PIL import Image
np.random.seed(409570391)

In [3]:
#7項皮膚疾病簡稱與全名
lesion_type_dict = {
    'nv': 'Melanocytic nevi',
    'mel': 'Melanoma',
    'bkl': 'Benign keratosis-like lesions ',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma'
}


In [4]:
pd.Categorical(lesion_type_dict).codes


array([5, 4, 2, 1, 0, 6, 3], dtype=int8)

In [5]:
# Download from Google Drive
!gdown --id 1orCwIp_avT4Yg9lCT8pjImwzqBg4_xiI
!unzip project03.zip

/usr/local/lib/python3.9/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1orCwIp_avT4Yg9lCT8pjImwzqBg4_xiI
To: /content/project03.zip
100% 26.6M/26.6M [00:00<00:00, 79.0MB/s]
Archive:  project03.zip
  inflating: project3_test.csv       
  inflating: project3_train.csv      


In [6]:
#讀取影像資料，28*28*3個像素值欄位(pixel0000-pixel2351) + 1個分類類別欄位label
load_img = pd.read_csv('project3_train.csv')

In [7]:
load_img.head()

,pixel0000,pixel0001,pixel0002,pixel0003,pixel0004,pixel0005,pixel0006,pixel0007,pixel0008,pixel0009,...,pixel2343,pixel2344,pixel2345,pixel2346,pixel2347,pixel2348,pixel2349,pixel2350,pixel2351,label
0,30,15,20,35,19,27,94,69,73,152,...,22,9,13,11,2,4,9,1,0,0
1,1,0,0,7,1,5,103,56,68,192,...,127,72,74,24,5,6,0,1,1,0
2,129,91,92,182,145,145,205,169,168,189,...,64,39,41,5,2,6,2,2,1,0
3,9,8,8,11,10,10,10,9,9,24,...,11,9,10,8,7,7,5,5,5,0
4,26,13,19,25,10,17,24,6,5,23,...,22,6,9,27,9,10,23,5,6,0


In [8]:
load_img.shape

(8008, 2353)

In [9]:
load_img.iloc[: , :-1].values


array([[ 30,  15,  20, ...,   9,   1,   0],
       [  1,   0,   0, ...,   0,   1,   1],
       [129,  91,  92, ...,   2,   2,   1],
       ...,
       [127, 101, 108, ..., 121, 108, 125],
       [157,  82,  86, ..., 210, 126, 130],
       [176, 149, 166, ..., 175, 142, 159]])

In [10]:
X_img , y_label = load_img.iloc[: , :-1].values , load_img.iloc[: , -1].values


In [11]:
#將串列轉成矩陣
X_img_train = np.asarray(X_img.tolist())

#將一維的數據，轉換成三維(長*寬*RGB三色)
X_img_train=X_img_train.reshape(X_img_train.shape[0],28,28,3)

In [12]:
#檢查學習資料的照片數量、尺寸大小、維度
print("train data:",'images:',X_img_train.shape," labels:",y_label.shape) 

train data: images: (8008, 28, 28, 3)  labels: (8008,)


In [13]:
#標準化: 同除255(因為image的數字是0~255)
#X_img_train_normalize = X_img_train.astype('float32') / 255.0

In [14]:
#使用np_utils.to_categorical()傳入各參數的label標籤欄位，再執行OneHot encoding (轉成0或1的組合)
#y_label_train_OneHot = np_utils.to_categorical(y_label)

In [15]:
#檢查標籤總共有多少種分類
#這裡是共8008筆資料，每筆是7個0或1的組合
#y_label_train_OneHot.shape

建立與訓練CNN模型


In [16]:
#匯入keras中的Sequential、layers模組(Dense、 Dropout、 Activation、 Flatten、Conv2D、 MaxPooling2D、 ZeroPadding2D)
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D

In [17]:
# Design your CNN model\

#建立keras的Sequential模型
model_cnn = Sequential()

In [18]:
model_cnn.add(Conv2D(filters=32,kernel_size=(3,3),input_shape=(28,28,3),
                     activation='relu',padding='same'))

model_cnn.add(Conv2D(filters=32,kernel_size=(3,3),
                     activation='relu',padding='same'))
model_cnn.add(Conv2D(filters=32,kernel_size=(3,3),
                     activation='relu',padding='same'))

model_cnn.add(MaxPooling2D(pool_size=(2,2)))



model_cnn.add(Conv2D(filters=64,kernel_size=(3,3),
                     activation='relu',padding='same'))
model_cnn.add(Conv2D(filters=64,kernel_size=(3,3),
                     activation='relu',padding='same'))
model_cnn.add(MaxPooling2D(pool_size=(2,2)))


model_cnn.add(Conv2D(filters=128,kernel_size=(3,3),
                     activation='relu',padding='same'))
model_cnn.add(Conv2D(filters=128,kernel_size=(3,3),
                     activation='relu',padding='same'))
model_cnn.add(MaxPooling2D(pool_size=(2,2)))


model_cnn.add(Conv2D(filters=256,kernel_size=(3,3),
                     activation='relu',padding='same'))
model_cnn.add(Conv2D(filters=256,kernel_size=(3,3),
                     activation='relu',padding='same'))
model_cnn.add(MaxPooling2D(pool_size=(2,2)))



model_cnn.add(Flatten())
model_cnn.add(Dense(64,activation='relu'))
model_cnn.add(Dropout(0.3))
model_cnn.add(Dense(7,activation='softmax'))

In [19]:
print(model_cnn.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 32)        9248      
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 28, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 conv2d_3 (Conv2D)           (None, 14, 14, 64)        18496     
                                                                 
 conv2d_4 (Conv2D)           (None, 14, 14, 64)        36928     
                                                        

In [20]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
early_stopping = EarlyStopping(monitor='val_loss',patience=20, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss',factor=0.1,patience=10, verbose=1, min_lr=1e-6)

In [21]:
model_cnn.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])


In [24]:
epochs=150
batch_size=128

x_train, x_validate, y_train, y_validate = train_test_split(X_img_train_normalize, y_label_train_OneHot, test_size=0.2, shuffle=True)

#train_history=model_cnn.fit(x_train, y_train, batch_size=batch_size, validation_data=(x_validate,y_validate), epochs=epochs, verbose=1, callbacks=None)#callbacks=[early_stopping,reduce_lr]


NameError: ignored

In [ ]:
import matplotlib.pyplot as plt
def show_train_history(train_acc,test_acc, yAxisName):
  plt.plot(train_history.history[train_acc])
  plt.plot(train_history.history[test_acc])
  plt.title('Train History')
  plt.ylabel(yAxisName)
  plt.xlabel('Epoch')
  plt.legend(['train','val'], loc='upper left')
  plt.show


In [ ]:
show_train_history('accuracy','val_accuracy','Accuracy')


In [ ]:
show_train_history('loss','val_loss','Loss')


In [ ]:
scores = model_cnn.evaluate(x_validate, y_validate, verbose=0)
scores[1]

In [ ]:
prediction = np.argmax(model_cnn.predict(x_validate), axis=-1)
pd.crosstab(np.argmax(y_validate, axis=-1), prediction, rownames=['label'], colnames=['predict'])

In [ ]:
# 使用最後的模型進行測試資料預測
load_test_img = pd.read_csv('project3_test.csv')
img_test = load_test_img.values

In [ ]:
x_test=img_test.reshape(img_test.shape[0],28,28,3)
x_test_normalize = x_test.astype('float32') / 255.0

In [ ]:
df_submit = pd.DataFrame([], columns=['Id', 'Label'])
df_submit['Id'] = [f'{i:04d}' for i in range(len(x_test_normalize))]
df_submit['Label'] = np.argmax(model_cnn.predict(x_test_normalize), axis=-1)

In [ ]:
df_submit.to_csv('submission_CNN7.csv', index=None)


In [ ]:
x_img_train, x_img_test, y_label_train, y_label_test = train_test_split(X_img_train_normalize, y_label_train_OneHot, test_size=0.2, random_state=2)


In [ ]:
x_img_train_normalize = x_img_train.astype('float32') / 255.0
x_img_test_normalize = x_img_test.astype('float32') / 255.0